In [1]:
# for deepagents to have access to ANTHROPIC_API_KEY
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import logging

from deepagents import create_deep_agent
from langchain.tools.base import BaseTool
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import InMemorySaver

logger = logging.getLogger(__name__)
MCP_SERVERS = {
    "gis_search": {
        "url": "http://localhost:8081/mcp",
        "transport": "streamable_http",
    }
}

MCP_CLIENT = MultiServerMCPClient(MCP_SERVERS)  # type: ignore [arg-type]

MCP_TOOLS: list[BaseTool] = await MCP_CLIENT.get_tools()

In [3]:
[t.name for t in MCP_TOOLS]

['gis_layer_search']

In [4]:
instructions = "You are the GIS team's interface to the GIS content semantic search tool. Use the tool to help the user find relevant GIS content. You can use the tool as many times as you'd like throughout the conversation, so feel free to do follow-up searches, ask clarifying questions of the user, etc."

checkpointer = InMemorySaver()

agent = create_deep_agent(
    MCP_TOOLS,
    instructions,
    checkpointer=checkpointer,
)# .with_config({"configurable": {"recursion_limit": 2}})


question = "I'm looking for parcel data for the two following cities: Sanford, FL (Seminole County) and Kelso, WA (Cowlitz County). Can you help me find them?"
datas = []

async for data in agent.astream(
    {
        "messages": [
            {
                "role": "user",
                "content": question,
            }
        ]
    },
    stream_mode="updates",
    config={"configurable": {"thread_id": "1"}},
):
    datas.append(data)
    print(data)

# await agent.aget_state({"configurable": {"thread_id": "1"}})


{'agent': {'messages': [AIMessage(content=[{'text': "I'll help you find parcel data for both Sanford, FL and Kelso, WA. Let me search for parcel data in both locations.", 'type': 'text'}, {'id': 'toolu_01HWZPpgKrjPyUD2XdKSeUGa', 'input': {'todos': [{'content': 'Search for parcel data in Sanford, FL (Seminole County)', 'status': 'pending'}, {'content': 'Search for parcel data in Kelso, WA (Cowlitz County)', 'status': 'pending'}]}, 'name': 'write_todos', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_0124oja1NbmSKPVVUjPJQX7c', 'model': 'claude-sonnet-4-20250514', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 5305, 'output_tokens': 150, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-20250514'}, id='run--5e3502c8-787b-4f2f-9034-ceef3c0c8e90-0', tool_c

In [5]:
state = await agent.aget_state({"configurable": {"thread_id": "1"}})

In [10]:
from IPython.display import Markdown

In [11]:
Markdown(state.values['messages'][-1].content)

Great! I found parcel data for both locations. Here's what I discovered:

## **Sanford, FL (Seminole County) Parcel Data:**

I found excellent parcel data for Seminole County, which includes Sanford. The main resource is:

**SEMINOLE County Parcels** (2 versions available):
- **Source**: Florida Department of Transportation (FDOT) 
- **Data**: Comprehensive parcel data acquired from Seminole County and merged with tax roll data from Florida DOR
- **URLs**: 
  - Map Service: https://gis.fdot.gov/arcgis/rest/services/Parcels/MapServer/57
  - Feature Service: https://gis.fdot.gov/arcgis/rest/services/Parcels/FeatureServer/57

**Key Features**: This dataset includes extensive attributes such as:
- Parcel numbers and IDs
- Property values (just value, assessed value, market value)
- Owner information (names, addresses)
- Property characteristics (land area, building details, construction year)
- Sales history
- Physical addresses
- Tax assessment information

**Contact**: Ana Nowak, 850-410-4365

---

## **Kelso, WA (Cowlitz County) Parcel Data:**

I found several parcel data sources for Cowlitz County, which includes Kelso:

**1. Cowlitz County Parcels** (Multiple sources):
- **Washington State DAHP**: https://wisaard.dahp.wa.gov/server/rest/services/County_Parcels/MapServer/8
- **Washington State DAHP Feature Service**: https://wisaard.dahp.wa.gov/server/rest/services/County_Parcels/FeatureServer/8

**2. Washington State County Tax Parcels**:
- **Source**: Washington DNR
- **URL**: https://gis.dnr.wa.gov/site2/rest/services/Public_Forest_Practices/WADNR_PUBLIC_OCIO_Parcels/MapServer/0

**3. Public Parcels 2019**:
- **URL**: https://services.arcgis.com/6lCKYNJLvwTXqrmp/ArcGIS/rest/services/Lacamas_Map/FeatureServer/25

**Important Note**: The Washington State parcel datasets mention that **owner names and mailing addresses are missing for Cowlitz County** since the county hasn't provided this information to the state. However, the datasets still contain parcel boundaries, situs addresses, land use codes, and property values.

**Key Features Available**:
- Parcel numbers and account numbers
- Site addresses
- Tax codes and land use codes  
- Property values (land and building values)
- Links to county assessor websites
- Parcel boundaries

Both datasets provide comprehensive parcel information, though the Florida data appears more complete with ownership details, while the Washington data has some limitations regarding owner information for Cowlitz County specifically.